## Gallup data processor (from xlsx to json)
v2 April 12th 2017


---
The Gallup data are downloaded from the Gallup Worl Poll website, the interface is not working so well and everything is done to prevent the user to easily recreate the database.. 

Gallup world poll data are stored in xlsx files, one variable per file. Each variable has differents answer modalities, country and time coverage. To map the information (name of the indicators, label, definition, answer modality etc.) a json file has been created (Gallup_modality.json). The raw files are stored with and ID number as name from gallup0 to gallup81. This information is displaid in the Gallup_modality.json file.

What we want is to create json files, 1 per country, containing all variables and their regional averages, similar income level and world averages.  

What this programme does:
- read excel file containing single Gallup Variables
- insert iso3, Region, Income_class codes
- compute and insert regional, income class, world average
- create single country csv files 
- write the corresponding json file


To function properly, the working folder contains two subfolders ("csv" and "json) to host temp files and the final json files. The working folder contains additionnal text files to store extr info:
- "json meta country.json.txt"
- "json meta country Gallup.txt"



## Gallup mapping all indicators

To produce dataviz we stored necessary instruction/information in a separate json file (dataviz_production.json). The following script shows how to retrieve this information

In [ ]:
import os
from IPython.display import HTML
import json
from string import Template
import winsound


#=============================== Uncomplete file to generate specific dataviz not all at one time ==================
#with open('dataviz_specific.json', encoding='utf-8') as data_file:data = json.load(data_file)

#=============================== complete mapping of dataviz to generate all of them at one time ==================
# Read production instructions from json file: dataviz_production.json that contains mapping of all dataviz
with open('dataviz_production.json', encoding='utf-8') as data_file:data = json.load(data_file)
indicatorlist=data["GALLUP"]["indicator_modality"]
for ind in indicatorlist:
    print(ind, "| modality:",  data["GALLUP"]["indicator_modality"][ind]["modality"], "| duration:", data["GALLUP"]["indicator_modality"][ind]["duration"])

## Reshaphing all Gallup CSV files 2.0

This first script reads all gallup single variable files, compute the differents averages and write single csv files

In [ ]:
import pandas as pd
from pandas import Series, DataFrame, concat
from datetime import datetime
import numpy as np
import winsound
import os
import csv
import json

#Gallup 82 excel files are stored in: 
folder='data/Gallup/'

#=================== Meta information we will use in this programmme =============================
# Country in Gallup WP and their associeted iso3
country_dict_GALLUP = {"Afghanistan":"AFG", "Albania":"ALB", "Algeria":"DZA", "Angola":"AGO", "Argentina":"ARG", "Armenia":"ARM", "Australia":"AUS", "Austria":"AUT", "Azerbaijan":"AZE", "Bahrain":"BHR", "Bangladesh":"BGD", "Belarus":"BLR", "Belgium":"BEL", "Belize":"BLZ", "Benin":"BEN", "Bhutan":"BTN", "Bolivia":"BOL", "Bosnia and Herzegovina":"BIH", "Botswana":"BWA", "Brazil":"BRA", "Bulgaria":"BGR", "Burkina Faso":"BFA", "Burundi":"BDI", "Cambodia":"KHM", "Cameroon":"CMR", "Canada":"CAN", "Central African Republic":"CAF", "Chad":"TCD", "Chile":"CHL", "China":"CHN", "Colombia":"COL", "Comoros":"COM", "Congo (Kinshasa)":"COD", "Congo Brazzaville":"COG", "Costa Rica":"CRI", "Cote d'Ivoire":"CIV", "Croatia":"HRV", "Cuba":"CUB", "Cyprus":"CYP", "Czech Republic":"CZE", "Denmark":"DNK", "Djibouti":"DJI", "Dominican Republic":"DOM", "Ecuador":"ECU", "Egypt":"EGY", "El Salvador":"SLV", "Estonia":"EST", "Ethiopia":"ETH", "Finland":"FIN", "France":"FRA", "Gabon":"GAB", "Georgia":"GEO", "Germany":"DEU", "Ghana":"GHA", "Greece":"GRC", "Guatemala":"GTM", "Guinea":"GIN", "Guyana":"GUY", "Haiti":"HTI", "Honduras":"HND", "Hong Kong":"HKG", "Hungary":"HUN", "Iceland":"ISL", "India":"IND", "Indonesia":"IDN", "Iran":"IRN", "Iraq":"IRQ", "Ireland":"IRL", "Israel":"ISR", "Italy":"ITA", "Jamaica":"JAM", "Japan":"JPN", "Jordan":"JOR", "Kazakhstan":"KAZ", "Kenya":"KEN", "Kosovo":"UNK", "Kuwait":"KWT", "Kyrgyzstan":"KGZ", "Lao People's Democratic Republic":"LAO", "Latvia":"LVA", "Lebanon":"LBN", "Lesotho":"LSO", "Liberia":"LBR", "Libya":"LBY", "Lithuania":"LTU", "Luxembourg":"LUX", "Macedonia":"MKD", "Madagascar":"MDG", "Malawi":"MWI", "Malaysia":"MYS", "Mali":"MLI", "Malta":"MLT", "Mauritania":"MRT", "Mauritius":"MUS", "Mexico":"MEX", "Moldova":"MDA", "Mongolia":"MNG", "Montenegro":"MNE", "Morocco":"MAR", "Mozambique":"MOZ", "Myanmar":"MMR", "Namibia":"NAM", "Nepal":"NPL", "Netherlands":"NLD", "New Zealand":"NZL", "Nicaragua":"NIC", "Niger":"NER", "Nigeria":"NGA", "Norway":"NOR", "Oman":"OMN", "Pakistan":"PAK", "Palestinian Territories":"PSE", "Panama":"PAN", "Paraguay":"PRY", "Peru":"PER", "Philippines":"PHL", "Poland":"POL", "Portugal":"PRT", "Puerto Rico":"PRI", "Qatar":"QAT", "Romania":"ROU", "Russia":"RUS", "Rwanda":"RWA", "Saudi Arabia":"SAU", "Senegal":"SEN", "Serbia":"SRB", "Sierra Leone":"SLE", "Singapore":"SGP", "Slovakia":"SVK", "Slovenia":"SVN", "Somalia":"SOM", "South Africa":"ZAF", "South Korea":"KOR", "South Sudan":"SSD", "Spain":"ESP", "Sri Lanka":"LKA", "Sudan":"SDN", "Suriname":"SUR", "Swaziland":"SWZ", "Sweden":"SWE", "Switzerland":"CHE", "Syria":"SYR", "Taiwan":"TWN", "Tajikistan":"TJK", "Tanzania":"TZA", "Thailand":"THA", "Togo":"TGO", "Trinidad and Tobago":"TTO", "Tunisia":"TUN", "Turkey":"TUR", "Turkmenistan":"TKM", "Uganda":"UGA", "Ukraine":"UKR", "United Arab Emirates":"ARE", "United Kingdom":"GBR", "United States of America":"USA", "Uruguay":"URY", "Uzbekistan":"UZB", "Venezuela":"VEN", "Vietnam":"VNM", "Yemen":"YEM", "Zambia":"ZMB", "Zimbabwe":"ZWE"}
# Region and income group associated with iso3
region_dict= {"ASM":"UMC,EAS,WLD","AUS":"OEC,EAS,WLD","BRN":"NOC,EAS,WLD","CHN":"UMC,EAS,WLD","FJI":"UMC,EAS,WLD","FSM":"LMC,EAS,WLD","GUM":"NOC,EAS,WLD","HKG":"NOC,EAS,WLD","IDN":"LMC,EAS,WLD","JPN":"OEC,EAS,WLD","KHM":"LIC,EAS,WLD","KIR":"LMC,EAS,WLD","KOR":"OEC,EAS,WLD","LAO":"LMC,EAS,WLD","MAC":"NOC,EAS,WLD","MHL":"UMC,EAS,WLD","MMR":"LMC,EAS,WLD","MNG":"UMC,EAS,WLD","MNP":"NOC,EAS,WLD","MYS":"UMC,EAS,WLD","NCL":"NOC,EAS,WLD","NZL":"OEC,EAS,WLD","PHL":"LMC,EAS,WLD","PLW":"UMC,EAS,WLD","PNG":"LMC,EAS,WLD","PRK":"LIC,EAS,WLD","PYF":"NOC,EAS,WLD","SGP":"NOC,EAS,WLD","SLB":"LMC,EAS,WLD","THA":"UMC,EAS,WLD","TLS":"LMC,EAS,WLD","TON":"UMC,EAS,WLD","TUV":"UMC,EAS,WLD","TWN":"NOC,EAS,WLD","VNM":"LMC,EAS,WLD","VUT":"LMC,EAS,WLD","WSM":"LMC,EAS,WLD","":"LMC,ECS,WLD","ALB":"UMC,ECS,WLD","AND":"NOC,ECS,WLD","ARM":"LMC,ECS,WLD","AUT":"OEC,ECS,WLD","AZE":"UMC,ECS,WLD","BEL":"OEC,ECS,WLD","BGR":"UMC,ECS,WLD","BIH":"UMC,ECS,WLD","BLR":"UMC,ECS,WLD","CHE":"OEC,ECS,WLD","CHI":"NOC,ECS,WLD","CYP":"NOC,ECS,WLD","CZE":"OEC,ECS,WLD","DEU":"OEC,ECS,WLD","DNK":"OEC,ECS,WLD","ESP":"OEC,ECS,WLD","EST":"OEC,ECS,WLD","FIN":"OEC,ECS,WLD","FRA":"OEC,ECS,WLD","FRO":"NOC,ECS,WLD","GBR":"OEC,ECS,WLD","GEO":"LMC,ECS,WLD","GRC":"OEC,ECS,WLD","GRL":"NOC,ECS,WLD","HRV":"NOC,ECS,WLD","HUN":"OEC,ECS,WLD","IMN":"NOC,ECS,WLD","IRL":"OEC,ECS,WLD","ISL":"OEC,ECS,WLD","ITA":"OEC,ECS,WLD","KAZ":"UMC,ECS,WLD","KGZ":"LMC,ECS,WLD","LIE":"NOC,ECS,WLD","LTU":"NOC,ECS,WLD","LUX":"OEC,ECS,WLD","LVA":"NOC,ECS,WLD","MCO":"NOC,ECS,WLD","MDA":"LMC,ECS,WLD","MKD":"UMC,ECS,WLD","MNE":"UMC,ECS,WLD","NLD":"OEC,ECS,WLD","NOR":"OEC,ECS,WLD","POL":"OEC,ECS,WLD","PRT":"OEC,ECS,WLD","ROU":"UMC,ECS,WLD","RUS":"NOC,ECS,WLD","SMR":"NOC,ECS,WLD","SRB":"UMC,ECS,WLD","SVK":"OEC,ECS,WLD","SVN":"OEC,ECS,WLD","SWE":"OEC,ECS,WLD","TJK":"LMC,ECS,WLD","TKM":"UMC,ECS,WLD","TUR":"UMC,ECS,WLD","UKR":"LMC,ECS,WLD","UZB":"LMC,ECS,WLD","ABW":"NOC,LCN,WLD","ARG":"NOC,LCN,WLD","ATG":"NOC,LCN,WLD","BHS":"NOC,LCN,WLD","BLZ":"UMC,LCN,WLD","BOL":"LMC,LCN,WLD","BRA":"UMC,LCN,WLD","BRB":"NOC,LCN,WLD","CHL":"OEC,LCN,WLD","COL":"UMC,LCN,WLD","CRI":"UMC,LCN,WLD","CUB":"UMC,LCN,WLD","CUW":"NOC,LCN,WLD","CYM":"NOC,LCN,WLD","DMA":"UMC,LCN,WLD","DOM":"UMC,LCN,WLD","ECU":"UMC,LCN,WLD","GRD":"UMC,LCN,WLD","GTM":"LMC,LCN,WLD","GUY":"LMC,LCN,WLD","HND":"LMC,LCN,WLD","HTI":"LIC,LCN,WLD","JAM":"UMC,LCN,WLD","KNA":"NOC,LCN,WLD","LCA":"UMC,LCN,WLD","MAF":"NOC,LCN,WLD","MEX":"UMC,LCN,WLD","NIC":"LMC,LCN,WLD","PAN":"UMC,LCN,WLD","PER":"UMC,LCN,WLD","PRI":"NOC,LCN,WLD","PRY":"UMC,LCN,WLD","SLV":"LMC,LCN,WLD","SUR":"UMC,LCN,WLD","SXM":"NOC,LCN,WLD","TCA":"NOC,LCN,WLD","TTO":"NOC,LCN,WLD","URY":"NOC,LCN,WLD","VCT":"UMC,LCN,WLD","VEN":"NOC,LCN,WLD","VIR":"NOC,LCN,WLD","ARE":"NOC,MEA,WLD","BHR":"NOC,MEA,WLD","DJI":"LMC,MEA,WLD","DZA":"UMC,MEA,WLD","EGY":"LMC,MEA,WLD","IRN":"UMC,MEA,WLD","IRQ":"UMC,MEA,WLD","ISR":"OEC,MEA,WLD","JOR":"UMC,MEA,WLD","KWT":"NOC,MEA,WLD","LBN":"UMC,MEA,WLD","LBY":"UMC,MEA,WLD","MAR":"LMC,MEA,WLD","MLT":"NOC,MEA,WLD","OMN":"NOC,MEA,WLD","PSE":"LMC,MEA,WLD","QAT":"NOC,MEA,WLD","SAU":"NOC,MEA,WLD","SYR":"LMC,MEA,WLD","TUN":"UMC,MEA,WLD","YEM":"LMC,MEA,WLD","BMU":"NOC,NAC,WLD","CAN":"OEC,NAC,WLD","USA":"OEC,NAC,WLD","AFG":"LIC,SAS,WLD","BGD":"LMC,SAS,WLD","BTN":"LMC,SAS,WLD","IND":"LMC,SAS,WLD","LKA":"LMC,SAS,WLD","MDV":"UMC,SAS,WLD","NPL":"LIC,SAS,WLD","PAK":"LMC,SAS,WLD","AGO":"UMC,SSF,WLD","BDI":"LIC,SSF,WLD","BEN":"LIC,SSF,WLD","BFA":"LIC,SSF,WLD","BWA":"UMC,SSF,WLD","CAF":"LIC,SSF,WLD","CIV":"LMC,SSF,WLD","CMR":"LMC,SSF,WLD","COD":"LIC,SSF,WLD","COG":"LMC,SSF,WLD","COM":"LIC,SSF,WLD","CPV":"LMC,SSF,WLD","ERI":"LIC,SSF,WLD","ETH":"LIC,SSF,WLD","GAB":"UMC,SSF,WLD","GHA":"LMC,SSF,WLD","GIN":"LIC,SSF,WLD","GMB":"LIC,SSF,WLD","GNB":"LIC,SSF,WLD","GNQ":"NOC,SSF,WLD","KEN":"LMC,SSF,WLD","LBR":"LIC,SSF,WLD","LSO":"LMC,SSF,WLD","MDG":"LIC,SSF,WLD","MLI":"LIC,SSF,WLD","MOZ":"LIC,SSF,WLD","MRT":"LMC,SSF,WLD","MUS":"UMC,SSF,WLD","MWI":"LIC,SSF,WLD","NAM":"UMC,SSF,WLD","NER":"LIC,SSF,WLD","NGA":"LMC,SSF,WLD","RWA":"LIC,SSF,WLD","SDN":"LMC,SSF,WLD","SEN":"LMC,SSF,WLD","SLE":"LIC,SSF,WLD","SOM":"LIC,SSF,WLD","SSD":"LIC,SSF,WLD","STP":"LMC,SSF,WLD","SWZ":"LMC,SSF,WLD","SYC":"NOC,SSF,WLD","TCD":"LIC,SSF,WLD","TGO":"LIC,SSF,WLD","TZA":"LIC,SSF,WLD","UGA":"LIC,SSF,WLD","ZAF":"UMC,SSF,WLD","ZMB":"LMC,SSF,WLD","ZWE":"LIC,SSF,WLD"}

#Loop over the excel files
for n in range(0,81+1):   
    #===============================================================================================================================
    # Part 1 : Load and clean the data
    #===============================================================================================================================   
    
    # Open the file Gallup excel file
    dataset = pd.read_excel(folder+'gallup'+str(n)+'.xlsx')
    
    #Read and save label and definition
    label=dataset["Unnamed: 1"][dataset["GALLUP"] == "Short Text:"].tolist()
    label=label[0]
    definition=dataset["Unnamed: 1"][dataset["GALLUP"] == "Long Text:"].tolist()
    definition=definition[0]
    print(" Proceed with File:",n, "\n Label:",label,'\n', "Definition:", definition )
    #Store column names existing in the xlsx file
    col=[dataset.columns.values]

    #Retrieve the real modalities (possible answers) and var names that are stored in the xlsx file:
    #store index of row that contain the actual label
    location=dataset[dataset['GALLUP'] == 'Geography'].index.tolist()
    #actual column names stored in the dataset
    newcol=dataset.ix[location[0]]
    
    #replace current column names by real column names - whatever their numbers
    for a in range (0,len(dataset.columns.values)):
        for column in col:
            # change columns name : column[a],"->", newcol[a]
            dataset = dataset.rename(columns={column[a]:newcol[a]})

    #change column names (Gallup uses Geography for country and Time for year
    dataset = dataset.rename(columns={'Geography':'cname', 'Time':'year'})
    
    #clean data => suppress empty lines
    dataset=dataset.drop(dataset.index[:location[0]+1])
    
    #We will add meta informations for countries (iso3, income group and Region) we will add empty column to host this information
    dataset.insert(0, 'iso3', np.nan)
    dataset.insert(0, 'Income_class', np.nan)
    dataset.insert(0, 'Region', np.nan)

    #set iso3 according to cname
    for cname,iso in country_dict_GALLUP.items():
        dataset.loc[dataset['cname']==cname, 'iso3'] = iso

    #set iso3 according to cname
    for iso,region_iso in region_dict.items():
        dataset.loc[dataset['iso3']==iso, 'Region'] = region_iso.split(',')[1]
        dataset.loc[dataset['iso3']==iso, 'Income_class'] = region_iso.split(',')[0]
    
    #Save the clean dataset as a csv  
    #dataset.to_csv(folder+'csv/'+label+'.csv')

    #For our needs we will here juste use the demographic "Aggregate"
    # We then create a subset of the data corresponding to this

    dataset_agg=dataset[(dataset["Demographic"]=="Age") | (dataset["Demographic"]=="Gender") | (dataset["Demographic"]=="Urban / Rural")  |(dataset["Demographic"]=="Aggregate") ] #& dataset[dataset["Demographic"]=="Age"]
    

    #we will not neet the sample size, nor the Demographic values
    try:
        dataset_agg = dataset_agg.drop(['Demographic','N Size', 'DK/RF'], 1)
    except:
        dataset_agg = dataset_agg.drop(['Demographic','N Size'], 1)
    #We use a double index as iso3 and year
    dataset_agg  = dataset_agg.set_index(["iso3", "year"])
    #Save the subset dataset as csv using  the label into the file name
    #print(dataset_agg.head())
    dataset_agg.to_csv(folder+'csv/'+label+'-agg.csv')

    #===============================================================================================================================
    # Part 2 : Computing group averages (Region, Income group and World) 
    #===============================================================================================================================
    
    #Load the aggregated dataset stored in a csv
    df = pd.read_csv(folder+'csv/'+label+'-agg.csv')
 
    #Compute average by income for all indicators in the DataFrame
    income= df.groupby(['Income_class', 'year']).agg([np.mean])
    #Compute average by region for all indicators in the DataFrame 
    region= df.groupby(['Region', 'year']).agg([np.mean])
    #compute average by demographic
    demographic= df.groupby(['Demographic Value', 'year']).agg([np.mean])
    #compute world average
    world= df.groupby(['year']).agg([np.mean])
    
    #!! world is not multiindexed here, it only has year, we thus need to create a multi index containing iso3= WLD
    yearRange=[]
    isolistW=[]
    #Save as list dates ex 1980, 1981, ... 2020
    yearRange=world.index.values
    #Save as list  'WLD' time the number of values in world.index.values
    for item in world.index.values:
        isolistW.append('WLD')
    #Redindex World as (WLD,year) eg (WLD, 1980)...(WLD,2020)
    world.index= zip(isolistW, yearRange)
    world.index.name=['iso3', 'year']
    
    #Concatenate income, region averages and world average
    df = concat([demographic, income,region,world])
    
    #Store it in a csv in case we need it later
    df.to_csv(folder+'csv/'+label+'-agg_mean.csv')
    
    #open it again (help reindexing)
    df = pd.read_csv(folder+'csv/'+label+'-agg_mean.csv')
    df= df.rename(columns={'Unnamed: 0':'iso3', 'Unnamed: 1':'year', 'Unnamed: 2':'Demographic'})
    df=df.drop(df.index[[0,1]])
    df=df.set_index(["iso3", "year"])

    #Now we have a dataframe containing the data and one containing the averages: lets concatenate them to have a single dataframe
    dataset_agg=dataset_agg.append(df)
    dataset_agg["label"]=label
    #Store it as a csv
    dataset_agg.to_csv(folder+'csv/agg_final'+str(n)+'.csv')
    #.... Done with the averages
    
#=== Done: victory message and song ===
print("\n*******************# Job Well done ! #********************\n")
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

## Create final dataset containing all data
This script concatenate the single files into one (too) big csv file

In [ ]:
import pandas as pd
from pandas import Series, DataFrame, concat
from datetime import datetime
import numpy as np
import winsound
import os
import csv

folder='data/Gallup/'
frame = dict()
final=DataFrame()
for n in range(0,81+1):
    frame[n]=pd.read_csv(folder+'csv/agg_final'+str(n)+'.csv')
    final = concat([final, frame[n]])
final.to_csv(folder+'final_GALLUP.csv')
#=== Done: victory message and song ===

print("\n*******************# Job Well done ! #********************\n")
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

## Create each country files out of complete dataset
This script write separate country files containing all variables and their averages. These files are create in order to shorten the time necessary to write the json. NB. Using one single big file it takes over 40 minutes to create each json, using smaller files, the processing time drop to 3 minutes..

In [ ]:
from pandas import Series, DataFrame, concat
from datetime import datetime
import numpy as np
import winsound
import os
import csv
import pandas as pd
import json
from os import listdir
from os.path import isfile, join
import logging

# Country in Gallup WP and their associeted iso3
#test list: {"Afghanistan":"CYP,LIC,SAS,WLD","Afghanistan":"AFG,LIC,SAS,WLD","Afghanistan":"BOL,LIC,SAS,WLD"}
country_dict_GALLUP ={"Afghanistan":"AFG,LIC,SAS,WLD","Albania":"ALB,UMC,ECS,WLD","Algeria":"DZA,UMC,MEA,WLD","Angola":"AGO,UMC,SSF,WLD","Argentina":"ARG,NOC,LCN,WLD","Armenia":"ARM,LMC,ECS,WLD","Australia":"AUS,OEC,EAS,WLD","Austria":"AUT,OEC,ECS,WLD","Azerbaijan":"AZE,UMC,ECS,WLD","Bahrain":"BHR,NOC,MEA,WLD","Bangladesh":"BGD,LMC,SAS,WLD","Belarus":"BLR,UMC,ECS,WLD","Belgium":"BEL,OEC,ECS,WLD","Belize":"BLZ,UMC,LCN,WLD","Benin":"BEN,LIC,SSF,WLD","Bhutan":"BTN,LMC,SAS,WLD","Bolivia":"BOL,LMC,LCN,WLD","Bosnia and Herzegovina":"BIH,UMC,ECS,WLD","Botswana":"BWA,UMC,SSF,WLD","Brazil":"BRA,UMC,LCN,WLD","Bulgaria":"BGR,UMC,ECS,WLD","Burkina Faso":"BFA,LIC,SSF,WLD","Burundi":"BDI,LIC,SSF,WLD","Cambodia":"KHM,LIC,EAS,WLD","Cameroon":"CMR,LMC,SSF,WLD","Canada":"CAN,OEC,NAC,WLD","Central African Republic":"CAF,LIC,SSF,WLD","Chad":"TCD,LIC,SSF,WLD","Chile":"CHL,OEC,LCN,WLD","China":"CHN,UMC,EAS,WLD","Colombia":"COL,UMC,LCN,WLD","Comoros":"COM,LIC,SSF,WLD","Congo (Kinshasa)":"COD,LIC,SSF,WLD","Congo Brazzaville":"COG,LMC,SSF,WLD","Costa Rica":"CRI,UMC,LCN,WLD","Cote d'Ivoire":"CIV,LMC,SSF,WLD","Croatia":"HRV,NOC,ECS,WLD","Cuba":"CUB,UMC,LCN,WLD","Cyprus":"CYP,NOC,ECS,WLD","Czech Republic":"CZE,OEC,ECS,WLD","Denmark":"DNK,OEC,ECS,WLD","Djibouti":"DJI,LMC,MEA,WLD","Dominican Republic":"DOM,UMC,LCN,WLD","Ecuador":"ECU,UMC,LCN,WLD","Egypt":"EGY,LMC,MEA,WLD","El Salvador":"SLV,LMC,LCN,WLD","Estonia":"EST,OEC,ECS,WLD","Ethiopia":"ETH,LIC,SSF,WLD","Finland":"FIN,OEC,ECS,WLD","France":"FRA,OEC,ECS,WLD","Gabon":"GAB,UMC,SSF,WLD","Georgia":"GEO,LMC,ECS,WLD","Germany":"DEU,OEC,ECS,WLD","Ghana":"GHA,LMC,SSF,WLD","Greece":"GRC,OEC,ECS,WLD","Guatemala":"GTM,LMC,LCN,WLD","Guinea":"GIN,LIC,SSF,WLD","Guyana":"GUY,LMC,LCN,WLD","Haiti":"HTI,LIC,LCN,WLD","Honduras":"HND,LMC,LCN,WLD","Hong Kong":"HKG,NOC,EAS,WLD","Hungary":"HUN,OEC,ECS,WLD","Iceland":"ISL,OEC,ECS,WLD","India":"IND,LMC,SAS,WLD","Indonesia":"IDN,LMC,EAS,WLD","Iran":"IRN,UMC,MEA,WLD","Iraq":"IRQ,UMC,MEA,WLD","Ireland":"IRL,OEC,ECS,WLD","Israel":"ISR,OEC,MEA,WLD","Italy":"ITA,OEC,ECS,WLD","Jamaica":"JAM,UMC,LCN,WLD","Japan":"JPN,OEC,EAS,WLD","Jordan":"JOR,UMC,MEA,WLD","Kazakhstan":"KAZ,UMC,ECS,WLD","Kenya":"KEN,LMC,SSF,WLD","Kosovo":"UNK,,,WLD","Kuwait":"KWT,NOC,MEA,WLD","Kyrgyzstan":"KGZ,LMC,ECS,WLD","Lao People's Democratic Republic":"LAO,LMC,EAS,WLD","Latvia":"LVA,NOC,ECS,WLD","Lebanon":"LBN,UMC,MEA,WLD","Lesotho":"LSO,LMC,SSF,WLD","Liberia":"LBR,LIC,SSF,WLD","Libya":"LBY,UMC,MEA,WLD","Lithuania":"LTU,NOC,ECS,WLD","Luxembourg":"LUX,OEC,ECS,WLD","Macedonia":"MKD,UMC,ECS,WLD","Madagascar":"MDG,LIC,SSF,WLD","Malawi":"MWI,LIC,SSF,WLD","Malaysia":"MYS,UMC,EAS,WLD","Mali":"MLI,LIC,SSF,WLD","Malta":"MLT,NOC,MEA,WLD","Mauritania":"MRT,LMC,SSF,WLD","Mauritius":"MUS,UMC,SSF,WLD","Mexico":"MEX,UMC,LCN,WLD","Moldova":"MDA,LMC,ECS,WLD","Mongolia":"MNG,UMC,EAS,WLD","Montenegro":"MNE,UMC,ECS,WLD","Morocco":"MAR,LMC,MEA,WLD","Mozambique":"MOZ,LIC,SSF,WLD","Myanmar":"MMR,LMC,EAS,WLD","Namibia":"NAM,UMC,SSF,WLD","Nepal":"NPL,LIC,SAS,WLD","Netherlands":"NLD,OEC,ECS,WLD","New Zealand":"NZL,OEC,EAS,WLD","Nicaragua":"NIC,LMC,LCN,WLD","Niger":"NER,LIC,SSF,WLD","Nigeria":"NGA,LMC,SSF,WLD","Norway":"NOR,OEC,ECS,WLD","Oman":"OMN,NOC,MEA,WLD","Pakistan":"PAK,LMC,SAS,WLD","Palestinian Territories":"PSE,LMC,MEA,WLD","Panama":"PAN,UMC,LCN,WLD","Paraguay":"PRY,UMC,LCN,WLD","Peru":"PER,UMC,LCN,WLD","Philippines":"PHL,LMC,EAS,WLD","Poland":"POL,OEC,ECS,WLD","Portugal":"PRT,OEC,ECS,WLD","Puerto Rico":"PRI,NOC,LCN,WLD","Qatar":"QAT,NOC,MEA,WLD","Romania":"ROU,UMC,ECS,WLD","Russia":"RUS,NOC,ECS,WLD","Rwanda":"RWA,LIC,SSF,WLD","Saudi Arabia":"SAU,NOC,MEA,WLD","Senegal":"SEN,LMC,SSF,WLD","Serbia":"SRB,UMC,ECS,WLD","Sierra Leone":"SLE,LIC,SSF,WLD","Singapore":"SGP,NOC,EAS,WLD","Slovakia":"SVK,OEC,ECS,WLD","Slovenia":"SVN,OEC,ECS,WLD","Somalia":"SOM,LIC,SSF,WLD","South Africa":"ZAF,UMC,SSF,WLD","South Korea":"KOR,OEC,EAS,WLD","South Sudan":"SSD,LIC,SSF,WLD","Spain":"ESP,OEC,ECS,WLD","Sri Lanka":"LKA,LMC,SAS,WLD","Sudan":"SDN,LMC,SSF,WLD","Suriname":"SUR,UMC,LCN,WLD","Swaziland":"SWZ,LMC,SSF,WLD","Sweden":"SWE,OEC,ECS,WLD","Switzerland":"CHE,OEC,ECS,WLD","Syria":"SYR,LMC,MEA,WLD","Taiwan":"TWN,NOC,EAS,WLD","Tajikistan":"TJK,LMC,ECS,WLD","Tanzania":"TZA,LIC,SSF,WLD","Thailand":"THA,UMC,EAS,WLD","Togo":"TGO,LIC,SSF,WLD","Trinidad and Tobago":"TTO,NOC,LCN,WLD","Tunisia":"TUN,UMC,MEA,WLD","Turkey":"TUR,UMC,ECS,WLD","Turkmenistan":"TKM,UMC,ECS,WLD","Uganda":"UGA,LIC,SSF,WLD","Ukraine":"UKR,LMC,ECS,WLD","United Arab Emirates":"ARE,NOC,MEA,WLD","United Kingdom":"GBR,OEC,ECS,WLD","United States of America":"USA,OEC,NAC,WLD","Uruguay":"URY,NOC,LCN,WLD","Uzbekistan":"UZB,LMC,ECS,WLD","Venezuela":"VEN,NOC,LCN,WLD","Vietnam":"VNM,LMC,EAS,WLD","Yemen":"YEM,LMC,MEA,WLD","Zambia":"ZMB,LMC,SSF,WLD","Zimbabwe":"ZWE,LIC,SSF,WLD"}

 # Load dataset 
folder='data/Gallup/'
dataset=pd.read_csv(folder+'final_GALLUP.csv')

for cname,iso_list in country_dict_GALLUP.items():
    iso_c=iso_list[0:3]
    iso_r=iso_list[4:7]
    iso_i=iso_list[8:10]
    print(cname,iso_c,iso_r, iso_i)

    dataset_AFG=dataset[(dataset["iso3"]==iso_c) | (dataset["iso3"]==iso_r) | (dataset["iso3"]==iso_i) | \
                        (dataset["iso3"]=="15-29") | (dataset["iso3"]=="30-49") | (dataset["iso3"]=="50+") |\
                        (dataset["iso3"]=="Aggregate") | (dataset["iso3"]=="Female") | (dataset["iso3"]=="Male") |\
                        (dataset["iso3"]=="Rural") | (dataset["iso3"]=="Urban") |(dataset["iso3"]=="WLD")]
                        
    dataset_AFG.to_csv(folder+'csv/Gallup_'+iso_c+'.csv')

print("\n*******************# Job Well done ! #********************\n")
winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

## Create JSON files from country csv

Finally this script write json files out of the country csv

In [ ]:
from pandas import Series, DataFrame, concat
from datetime import datetime
import numpy as np
import winsound
import os
import csv
import pandas as pd
import json
from os import listdir
from os.path import isfile, join
import logging

####################### SET LOG ################################
# Set logging parameters
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.getLogger().setLevel(logging.INFO)
# create a file handler
handler = logging.FileHandler('data/Gallup/LOG_json_gallup.log')
handler.setLevel(logging.INFO)
# add the handlers to the logger
logger.addHandler(handler)
################################################################

# Load dataset 
folder='data/Gallup/'

# Country in Gallup WP and their associeted iso3
#test list: {"Afghanistan":"CYP,LIC,SAS,WLD","Afghanistan":"AFG,LIC,SAS,WLD","Afghanistan":"BOL,LIC,SAS,WLD"}
country_dict_GALLUP1 ={"Afghanistan":"AFG,LIC,SAS,WLD","Albania":"ALB,UMC,ECS,WLD","Algeria":"DZA,UMC,MEA,WLD","Angola":"AGO,UMC,SSF,WLD","Argentina":"ARG,NOC,LCN,WLD","Armenia":"ARM,LMC,ECS,WLD","Australia":"AUS,OEC,EAS,WLD","Austria":"AUT,OEC,ECS,WLD","Azerbaijan":"AZE,UMC,ECS,WLD","Bahrain":"BHR,NOC,MEA,WLD","Bangladesh":"BGD,LMC,SAS,WLD","Belarus":"BLR,UMC,ECS,WLD","Belgium":"BEL,OEC,ECS,WLD","Belize":"BLZ,UMC,LCN,WLD","Benin":"BEN,LIC,SSF,WLD","Bhutan":"BTN,LMC,SAS,WLD","Bolivia":"BOL,LMC,LCN,WLD","Bosnia and Herzegovina":"BIH,UMC,ECS,WLD","Botswana":"BWA,UMC,SSF,WLD","Brazil":"BRA,UMC,LCN,WLD","Bulgaria":"BGR,UMC,ECS,WLD","Burkina Faso":"BFA,LIC,SSF,WLD","Burundi":"BDI,LIC,SSF,WLD","Cambodia":"KHM,LIC,EAS,WLD","Cameroon":"CMR,LMC,SSF,WLD","Canada":"CAN,OEC,NAC,WLD","Central African Republic":"CAF,LIC,SSF,WLD","Chad":"TCD,LIC,SSF,WLD","Chile":"CHL,OEC,LCN,WLD","China":"CHN,UMC,EAS,WLD","Colombia":"COL,UMC,LCN,WLD","Comoros":"COM,LIC,SSF,WLD","Congo (Kinshasa)":"COD,LIC,SSF,WLD","Congo Brazzaville":"COG,LMC,SSF,WLD","Costa Rica":"CRI,UMC,LCN,WLD","Cote d'Ivoire":"CIV,LMC,SSF,WLD","Croatia":"HRV,NOC,ECS,WLD","Cuba":"CUB,UMC,LCN,WLD","Cyprus":"CYP,NOC,ECS,WLD","Czech Republic":"CZE,OEC,ECS,WLD","Denmark":"DNK,OEC,ECS,WLD","Djibouti":"DJI,LMC,MEA,WLD","Dominican Republic":"DOM,UMC,LCN,WLD","Ecuador":"ECU,UMC,LCN,WLD","Egypt":"EGY,LMC,MEA,WLD","El Salvador":"SLV,LMC,LCN,WLD","Estonia":"EST,OEC,ECS,WLD","Ethiopia":"ETH,LIC,SSF,WLD","Finland":"FIN,OEC,ECS,WLD","France":"FRA,OEC,ECS,WLD","Gabon":"GAB,UMC,SSF,WLD","Georgia":"GEO,LMC,ECS,WLD","Germany":"DEU,OEC,ECS,WLD","Ghana":"GHA,LMC,SSF,WLD","Greece":"GRC,OEC,ECS,WLD","Guatemala":"GTM,LMC,LCN,WLD","Guinea":"GIN,LIC,SSF,WLD","Guyana":"GUY,LMC,LCN,WLD","Haiti":"HTI,LIC,LCN,WLD","Honduras":"HND,LMC,LCN,WLD","Hong Kong":"HKG,NOC,EAS,WLD","Hungary":"HUN,OEC,ECS,WLD","Iceland":"ISL,OEC,ECS,WLD","India":"IND,LMC,SAS,WLD","Indonesia":"IDN,LMC,EAS,WLD","Iran":"IRN,UMC,MEA,WLD","Iraq":"IRQ,UMC,MEA,WLD","Ireland":"IRL,OEC,ECS,WLD","Israel":"ISR,OEC,MEA,WLD","Italy":"ITA,OEC,ECS,WLD","Jamaica":"JAM,UMC,LCN,WLD","Japan":"JPN,OEC,EAS,WLD","Jordan":"JOR,UMC,MEA,WLD","Kazakhstan":"KAZ,UMC,ECS,WLD","Kenya":"KEN,LMC,SSF,WLD","Kosovo":"UNK,,,WLD","Kuwait":"KWT,NOC,MEA,WLD","Kyrgyzstan":"KGZ,LMC,ECS,WLD","Lao People's Democratic Republic":"LAO,LMC,EAS,WLD"}
country_dict_GALLUP2 ={"Latvia":"LVA,NOC,ECS,WLD","Lebanon":"LBN,UMC,MEA,WLD","Lesotho":"LSO,LMC,SSF,WLD","Liberia":"LBR,LIC,SSF,WLD","Libya":"LBY,UMC,MEA,WLD","Lithuania":"LTU,NOC,ECS,WLD","Luxembourg":"LUX,OEC,ECS,WLD","Macedonia":"MKD,UMC,ECS,WLD","Madagascar":"MDG,LIC,SSF,WLD","Malawi":"MWI,LIC,SSF,WLD","Malaysia":"MYS,UMC,EAS,WLD","Mali":"MLI,LIC,SSF,WLD","Malta":"MLT,NOC,MEA,WLD","Mauritania":"MRT,LMC,SSF,WLD","Mauritius":"MUS,UMC,SSF,WLD","Mexico":"MEX,UMC,LCN,WLD","Moldova":"MDA,LMC,ECS,WLD","Mongolia":"MNG,UMC,EAS,WLD","Montenegro":"MNE,UMC,ECS,WLD","Morocco":"MAR,LMC,MEA,WLD","Mozambique":"MOZ,LIC,SSF,WLD","Myanmar":"MMR,LMC,EAS,WLD","Namibia":"NAM,UMC,SSF,WLD","Nepal":"NPL,LIC,SAS,WLD","Netherlands":"NLD,OEC,ECS,WLD","New Zealand":"NZL,OEC,EAS,WLD","Nicaragua":"NIC,LMC,LCN,WLD","Niger":"NER,LIC,SSF,WLD","Nigeria":"NGA,LMC,SSF,WLD","Norway":"NOR,OEC,ECS,WLD","Oman":"OMN,NOC,MEA,WLD","Pakistan":"PAK,LMC,SAS,WLD","Palestinian Territories":"PSE,LMC,MEA,WLD","Panama":"PAN,UMC,LCN,WLD","Paraguay":"PRY,UMC,LCN,WLD","Peru":"PER,UMC,LCN,WLD","Philippines":"PHL,LMC,EAS,WLD","Poland":"POL,OEC,ECS,WLD","Portugal":"PRT,OEC,ECS,WLD","Puerto Rico":"PRI,NOC,LCN,WLD","Qatar":"QAT,NOC,MEA,WLD","Romania":"ROU,UMC,ECS,WLD","Russia":"RUS,NOC,ECS,WLD","Rwanda":"RWA,LIC,SSF,WLD","Saudi Arabia":"SAU,NOC,MEA,WLD","Senegal":"SEN,LMC,SSF,WLD","Serbia":"SRB,UMC,ECS,WLD","Sierra Leone":"SLE,LIC,SSF,WLD","Singapore":"SGP,NOC,EAS,WLD","Slovakia":"SVK,OEC,ECS,WLD","Slovenia":"SVN,OEC,ECS,WLD","Somalia":"SOM,LIC,SSF,WLD","South Africa":"ZAF,UMC,SSF,WLD","South Korea":"KOR,OEC,EAS,WLD","South Sudan":"SSD,LIC,SSF,WLD","Spain":"ESP,OEC,ECS,WLD","Sri Lanka":"LKA,LMC,SAS,WLD","Sudan":"SDN,LMC,SSF,WLD","Suriname":"SUR,UMC,LCN,WLD","Swaziland":"SWZ,LMC,SSF,WLD","Sweden":"SWE,OEC,ECS,WLD","Switzerland":"CHE,OEC,ECS,WLD","Syria":"SYR,LMC,MEA,WLD","Taiwan":"TWN,NOC,EAS,WLD","Tajikistan":"TJK,LMC,ECS,WLD","Tanzania":"TZA,LIC,SSF,WLD","Thailand":"THA,UMC,EAS,WLD","Togo":"TGO,LIC,SSF,WLD","Trinidad and Tobago":"TTO,NOC,LCN,WLD","Tunisia":"TUN,UMC,MEA,WLD","Turkey":"TUR,UMC,ECS,WLD","Turkmenistan":"TKM,UMC,ECS,WLD","Uganda":"UGA,LIC,SSF,WLD","Ukraine":"UKR,LMC,ECS,WLD","United Arab Emirates":"ARE,NOC,MEA,WLD","United Kingdom":"GBR,OEC,ECS,WLD","United States of America":"USA,OEC,NAC,WLD","Uruguay":"URY,NOC,LCN,WLD","Uzbekistan":"UZB,LMC,ECS,WLD","Venezuela":"VEN,NOC,LCN,WLD","Vietnam":"VNM,LMC,EAS,WLD","Yemen":"YEM,LMC,MEA,WLD","Zambia":"ZMB,LMC,SSF,WLD","Zimbabwe":"ZWE,LIC,SSF,WLD",}

# To generate all countries
country_dict_GALLUP = dict(country_dict_GALLUP1.items() | country_dict_GALLUP2.items())

indicatorlist= ["Confidence in Financial Institutions","Currently Own a Business","Economic Conditions",\
                "Feelings About Household Income","National Economy Getting Better",\
                "People Starting Businesses","Receive Money or Goods","Sent Financial Help","Standard of Living Better",\
                "Work Hard, Get Ahead","City Economy Getting Better","Count On to Help","Freedom in Your Life",\
                "Donated Money","Helped a Stranger","Local Job Market","Move Away or Stay",\
                "Move Permanently to Another Country","Overall City Satisfaction","Opportunities to Make Friends",\
                "Voiced Opinion to Official","Ideal Job","Volunteered Time","Freedom of Media","Home Has Access to Internet",\
                "Home Has Cellular Phone","Home Has Landline Telephone","Home Has Television","Children Learn and Grow",\
                "Children Respected","Education Level","Educational System","Racial Ethnic Minorities",\
                "Preserve the Environment","Quality of Air","Quality of Water","City: Good, Affordable Housing",\
                "Not Enough Money Food","Not Enough Money Shelter","Honesty of Elections","City Quality Healthcare",\
                "Have Enough Money","Health Problems","Feel Well-Rested","Felt Active and Productive",\
                "Learn or Do Something Interesting","Like What You Do Each Day","Worried About Money",\
                "Physical Health Near Perfect","Confidence in National Government","Confidence in Judicial System",\
                "Confidence in Local Police","Confidence in Military","Corruption in Government","Corruption Within Businesses",\
                "Money Property Stolen","Safe Walking Alone","Religion Important","Gay or Lesbian People",\
                "Good Place for Intellectually Disabled","Immigrants","Experience Anger Yesterday",\
                "Experience Sadness Yesterday","Experience Stress Yesterday","Experience Worry Yesterday",\
                "Experienced Enjoyment Yesterday","Life in Five Years","Life Today","Friends Family Give You Positive Energy",\
                "Recognition for Improving City or Area","Employment Status","Payroll to Population Index (P2P)","Unemployment Index"]

#Variable we decide not to keep
#["Employee Engagement","Recommend City","Age","Children Under 15","Urban Rural","City or Area is a Perfect Place","Religion", "Smile or Laugh","Economic Confidence Index"]

#Exceptions are variables which contains 5 modalities, ordered from 1... to 5.. we decided to treat them separately and keep only the extrema (1. and 5.)
exception_list=["City or Area is a Perfect Place","Have Enough Money","Felt Active and Productive",\
                "Learn or Do Something Interesting","Like What You Do Each Day","Worried About Money",\
                "Physical Health Near Perfect","Recognition for Improving City or Area", \
                "Friends Family Give You Positive Energy"]

#meta information to be addeed in the json
source='"data_source":{"source":"Gallup World Poll", "provider":"Gallup", "Download_date":"2016" }'

with open(folder+'Gallup_modality.json', encoding='utf-8') as data_file: 
    data = json.load(data_file)

# For each country
for cname,iso3 in country_dict_GALLUP.items():
    print('Processing:',cname)
    #only work on countries that have not yet been treated (in case program crashes to avoid doing everything from the begining)
    #onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    onlyfiles = [f for f in listdir(folder+'json/') if isfile(join(folder+'json/', f))]
    if 'GWP_'+iso3[0:3]+'.json' in onlyfiles:
        print("Already done:",cname,"... continue...")
        continue
    #Open country file
    dataset=pd.read_csv(folder+'csv/Gallup_'+iso3[0:3]+'.csv')
    # Clean dataset 
    try:
        dataset = file.drop('(DK)',1)
        datset=file.drop('(Refused)', 1)
        dataset=file.drop('Unnamed: 0', 1)
    except:
        pass

    #dataset.head()
    #Store year min and max of all data
    year_min_dataset=min(dataset['year'])
    year_max_dataset=max(dataset['year'])
    
    with open (folder+"json meta country.json.txt", "r") as meta:
        for line in meta :
            if line.startswith('"'+iso3[0:3]+'"'):
                metafinal=","+line+","+source
                
    file = open(folder+"temp.json", "w")
    file.write('{')
    
    # For each indicator
    for ind in indicatorlist:
        #Set Exception to 0
        Exception=0
        #Read the data description file to know how many modality exist for the current indicator
        nb_modality=data[ind]["nb_modality"]
        
        #Deal with exceptions
        if ind in exception_list:
            nb_modality=2
            Exception=1
        else:
            Exception=0
            
        #logger.info("country: "+cname+" | "+iso3[0:3]+"| Indicator: "+ind+" | Exception: "+str(Exception)) 
        #print("country:",cname, iso3[0:3],"Indicator:",ind,"Exception:",Exception)
        
        #Store the year vector for the indicator and country being treated 
        year_range=dataset["year"][(dataset["iso3"]==iso3[0:3])&(dataset["label"]==ind)]
        
        if len(year_range)==0 :
                temp_year_min=2006
                temp_year_max=2006
                logger.info(ind+ ": no data")
        else:
            temp_year_min=min(year_range)
            temp_year_max=max(year_range)
            
        #write the json
        file.write('"'+ind+'":{ "label": "'+ind+'", "definition": "","YearMin":'+str(temp_year_min)+',"YearMax":'+str(temp_year_max)+',')        
        
        for iso in iso3.split(','):
            file.write('"'+iso+'":{')
            
            # For each year
            for year in range (temp_year_min,temp_year_max+1):
                file.write('"'+str(year)+'":{')
                
                #for each modality
                for i in range(1,nb_modality+1):
                    modality=data[ind]['modality'+str(i)]
                    #deal with exception
                    if Exception==1 : 
                        if i==1: 
                            modality="1. Strongly disagree"
                        else:
                            modality="5. Strongly agree"    
                    
                    #logger.info("Porcessing: "+cname+', '+iso+', '+ind+','+str(year)+', '+modality+ ', i:'+str(i))
                    
                    if iso in ["LIC","LMC","NOC","OEC","UMC","EAS","ECS","LCN","MEA","NAC","SAS","SSF","WLD"]:
                                                
                        v_agg=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind) & (dataset["year"]==year)]
                        if len(v_agg) !=0:
                            v_agg=round(float(v_agg),2)
                        else:
                            v_agg='null'
                        
                        v_male=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind) & (dataset["year"]==year)]
                        if len(v_male) !=0:
                            v_male=round(float(v_male),2)
                        else:
                            v_male='null'
                        
                        v_female=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind) & (dataset["year"]==year)]
                        if len(v_female) !=0:
                            v_female=round(float(v_female),2)
                        else:
                            v_female='null'
                        
                        v_age1=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year)]
                        if len(v_age1) !=0:
                            v_age1=round(float(v_age1),2)
                        else:
                            v_age1='null'  

                        v_age2=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year)]
                        if len(v_age2) !=0:
                            v_age2=round(float(v_age2),2)
                        else:
                            v_age2='null'  

                        v_age3=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year)]
                        if len(v_age3) !=0:
                            v_age3=round(float(v_age3),2)
                        else:
                            v_age3='null'  

                        v_urban=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year)]
                        if len(v_urban) !=0:
                            v_urban=round(float(v_urban),2)
                        else:
                            v_urban='null'  

                        v_rural=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year)]
                        if len(v_rural) !=0:
                            v_rural=round(float(v_rural),2)
                        else:
                            v_rural='null'

                        #logger.info(v_agg,v_male,v_female,v_age1,v_age2,v_age3,v_urban,v_rural)
     
                    else:    
                        v_agg=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                 (dataset["Demographic Value"]=="Aggregate")]
                        if len(v_agg) !=0:
                            v_agg=float(v_agg)
                        else:
                            v_agg='null'  

                        v_male=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                (dataset["Demographic Value"]=="Male")]
                        if len(v_male) !=0:
                            v_male=float(v_male)
                        else:
                            v_male='null'  

                        v_female=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                (dataset["Demographic Value"]=="Female")]
                        if len(v_female) !=0:
                            v_female=float(v_female)
                        else:
                            v_female='null'  

                        v_age1=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                (dataset["Demographic Value"]=="15-29")]
                        if len(v_age1) !=0:
                            v_age1=float(v_age1)
                        else:
                            v_age1='null'  

                        v_age2=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                 (dataset["Demographic Value"]=="30-49")]
                        if len(v_age2) !=0:
                            v_age2=float(v_age2)
                        else:
                            v_age2='null'  

                        v_age3=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                 (dataset["Demographic Value"]=="50+")]
                        if len(v_age3) !=0:
                            v_age3=float(v_age3)
                        else:
                            v_age3='null'  

                        v_urban=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                (dataset["Demographic Value"]=="Urban")]
                        if len(v_urban) !=0:
                            v_urban=float(v_urban)
                        else:
                            v_urban='null'  

                        v_rural=dataset[modality][(dataset["iso3"]==iso)& (dataset["label"]==ind)& (dataset["year"]==year) & \
                                                (dataset["Demographic Value"]=="Rural")]
                        if len(v_rural) !=0:
                            v_rural=float(v_rural)
                        else:
                            v_rural='null'

                        #logger.info(v_agg,v_male,v_female,v_age1,v_age2,v_age3,v_urban,v_rural)

                    file.write('"'+modality+'":{"Aggregate":'+str(v_agg)+','+\
                                '"Male":'+str(v_male)+',"Female":'+str(v_female)+', "15-29":'+str(v_age1)+', "30-49":'+str(v_age2)+\
                                ', "50+":'+str(v_age3)+', "Urban":'+str(v_urban)+', "Rural":'+str(v_rural)+'}')
                    if i <nb_modality:
                        file.write(',') 

                    if (i==nb_modality) & (year!=temp_year_max):
                        file.write('},')
                        
                    if (i==nb_modality) & (year==temp_year_max) & (iso!="WLD"):
                        file.write('}},')
                        
                    if (iso=="WLD")& (i==nb_modality) & (year==temp_year_max) & (ind!=indicatorlist[-1:][0]):
                        file.write('}}},')
                        
                    if (iso=="WLD")& (i==nb_modality) & (year==temp_year_max) & (ind==indicatorlist[-1:][0]):
                        file.write('}}}'+metafinal+'}')

    file.close()
    file_in = open(folder+"temp.json", "r")
    file_out = open(folder+"json/"+"GWP_"+iso3[0:3]+".json", "wt")
    #nb we add ':' before 'nan' to avoid the label or definition text containing 'nan' to be changed by null...
    for line in file_in: file_out.write(line.replace(":nan", ":null") )
    file_in.close()
    file_out.close()
    print("\n*******************# Job Well done: "+cname+" #********************\n")
    winsound.PlaySound("SystemExit", winsound.SND_ALIAS)

In [ ]:
#use object retrieve information faster
def iso3_from_cname(x):
    return {"Afghanistan":"AFG,LIC,SAS,WLD","Albania":"ALB,UMC,ECS,WLD","Algeria":"DZA,UMC,MEA,WLD","Angola":"AGO,UMC,SSF,WLD","Argentina":"ARG,NOC,LCN,WLD","Armenia":"ARM,LMC,ECS,WLD","Australia":"AUS,OEC,EAS,WLD","Austria":"AUT,OEC,ECS,WLD","Azerbaijan":"AZE,UMC,ECS,WLD","Bahrain":"BHR,NOC,MEA,WLD","Bangladesh":"BGD,LMC,SAS,WLD","Belarus":"BLR,UMC,ECS,WLD","Belgium":"BEL,OEC,ECS,WLD","Belize":"BLZ,UMC,LCN,WLD","Benin":"BEN,LIC,SSF,WLD","Bhutan":"BTN,LMC,SAS,WLD","Bolivia":"BOL,LMC,LCN,WLD","Bosnia and Herzegovina":"BIH,UMC,ECS,WLD","Botswana":"BWA,UMC,SSF,WLD","Brazil":"BRA,UMC,LCN,WLD","Bulgaria":"BGR,UMC,ECS,WLD","Burkina Faso":"BFA,LIC,SSF,WLD","Burundi":"BDI,LIC,SSF,WLD","Cambodia":"KHM,LIC,EAS,WLD","Cameroon":"CMR,LMC,SSF,WLD","Canada":"CAN,OEC,NAC,WLD","Central African Republic":"CAF,LIC,SSF,WLD","Chad":"TCD,LIC,SSF,WLD","Chile":"CHL,OEC,LCN,WLD","China":"CHN,UMC,EAS,WLD","Colombia":"COL,UMC,LCN,WLD","Comoros":"COM,LIC,SSF,WLD","Congo (Kinshasa)":"COD,LIC,SSF,WLD","Congo Brazzaville":"COG,LMC,SSF,WLD","Costa Rica":"CRI,UMC,LCN,WLD","Cote d'Ivoire":"CIV,LMC,SSF,WLD","Croatia":"HRV,NOC,ECS,WLD","Cuba":"CUB,UMC,LCN,WLD","Cyprus":"CYP,NOC,ECS,WLD","Czech Republic":"CZE,OEC,ECS,WLD","Denmark":"DNK,OEC,ECS,WLD","Djibouti":"DJI,LMC,MEA,WLD","Dominican Republic":"DOM,UMC,LCN,WLD","Ecuador":"ECU,UMC,LCN,WLD","Egypt":"EGY,LMC,MEA,WLD","El Salvador":"SLV,LMC,LCN,WLD","Estonia":"EST,OEC,ECS,WLD","Ethiopia":"ETH,LIC,SSF,WLD","Finland":"FIN,OEC,ECS,WLD","France":"FRA,OEC,ECS,WLD","Gabon":"GAB,UMC,SSF,WLD","Georgia":"GEO,LMC,ECS,WLD","Germany":"DEU,OEC,ECS,WLD","Ghana":"GHA,LMC,SSF,WLD","Greece":"GRC,OEC,ECS,WLD","Guatemala":"GTM,LMC,LCN,WLD","Guinea":"GIN,LIC,SSF,WLD","Guyana":"GUY,LMC,LCN,WLD","Haiti":"HTI,LIC,LCN,WLD","Honduras":"HND,LMC,LCN,WLD","Hong Kong":"HKG,NOC,EAS,WLD","Hungary":"HUN,OEC,ECS,WLD","Iceland":"ISL,OEC,ECS,WLD","India":"IND,LMC,SAS,WLD","Indonesia":"IDN,LMC,EAS,WLD","Iran":"IRN,UMC,MEA,WLD","Iraq":"IRQ,UMC,MEA,WLD","Ireland":"IRL,OEC,ECS,WLD","Israel":"ISR,OEC,MEA,WLD","Italy":"ITA,OEC,ECS,WLD","Jamaica":"JAM,UMC,LCN,WLD","Japan":"JPN,OEC,EAS,WLD","Jordan":"JOR,UMC,MEA,WLD","Kazakhstan":"KAZ,UMC,ECS,WLD","Kenya":"KEN,LMC,SSF,WLD","Kosovo":"UNK,,,WLD","Kuwait":"KWT,NOC,MEA,WLD","Kyrgyzstan":"KGZ,LMC,ECS,WLD","Lao People's Democratic Republic":"LAO,LMC,EAS,WLD"}.get(x,"null")


iso3_from_cname('France')
